In [1]:
from collections import Counter
import requests
from nltk.corpus import stopwords
import string # "string" module is already installed with Python
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from datetime import datetime
import time
from requests.exceptions import ChunkedEncodingError, ConnectionError

In [2]:
#public key, so no need to hide
key = 'OSOegLs.PR2lwJ1dwCeje9vTj7FPOt3hvpYKtwKkhw'

### Retrieve information from Parliamentary Meetings (15. - 21. election period) (API)

In [3]:
wp=22
responses = []
end_date = datetime.today().strftime('%Y-%m-%d')

In [4]:
#Last executed at 2025-11-25 19:04:42 in 8m 37.45s

# max_attempts = 3
# while wp > 15:
#     url = f"https://search.dip.bundestag.de/api/v1/plenarprotokoll-text?f.zuordnung=BT&f.datum.end={end_date}&apikey={key}"

#     for attempt in range(1, max_attempts + 1):
#         try:
#             response = requests.get(url, timeout=30)
#             response.raise_for_status()
#             payload = response.json()
#             break
#         except (ChunkedEncodingError, ConnectionError) as exc:
#             print(f"Attempt {attempt}/{max_attempts} failed: {exc}")
#             if attempt == max_attempts:
#                 print("Giving up on this request, continuing with next loop iteration.")
#                 payload = None
#             else:
#                 time.sleep(2 * attempt)  # simple backoff
#         except requests.HTTPError as exc:
#             print(f"HTTP error {exc.response.status_code}: {exc}")
#             payload = None
#             break

#     if payload is None:
#         # Skip updating state for this iteration
#         continue

#     responses.append(payload)

#     docs = payload.get("documents", [])
#     if not docs:
#         print("No documents returned, stopping.")
#         break

#     dates = [doc["datum"] for doc in docs if "datum" in doc]
#     if not dates:
#         print("Documents missing 'datum', stopping.")
#         break

#     end_date = min(dates)
#     wp = docs[0]["wahlperiode"]

#     if wp <= 15:
#         break

!Will take about 8-10 minutes of fetching!

In [ ]:
# while wp > 15:   
#     url = f'https://search.dip.bundestag.de/api/v1/plenarprotokoll-text?f.zuordnung=BT&f.datum.end={end_date}&apikey={key}' 
#     response = requests.get(url).json()
#     responses.append(response)

#     date = []
    
#     for i in range(len(response['documents'])-1):
#         print(response['documents'][i]['datum'])
#         date.append(response['documents'][i]['datum'])
    
#     end_date = min(date)
    
#     wp = response['documents'][0]['wahlperiode']

#     if wp <= 15:
#         break 
        

2025-11-25
2025-11-14
2025-11-13
2025-11-12
2025-11-07
2025-11-06
2025-11-05
2025-10-17
2025-10-16
2025-10-16
2025-10-15
2025-10-10
2025-10-09
2025-10-08
2025-09-26
2025-09-25
2025-09-24
2025-09-23
2025-09-23
2025-09-18
2025-09-17
2025-09-16
2025-09-12
2025-09-11
2025-09-10
2025-07-11
2025-07-10
2025-07-10
2025-07-09
2025-07-08
2025-06-27
2025-06-26
2025-06-25
2025-06-24
2025-06-06
2025-06-05
2025-06-05
2025-06-04
2025-05-23
2025-05-22
2025-05-21
2025-05-16
2025-05-15
2025-05-14
2025-05-06
2025-05-06
2025-03-25
2025-03-18
2025-03-13
2025-02-11
2025-01-31
2025-01-30
2025-01-29
2024-12-20
2024-12-20
2024-12-19
2024-12-18
2024-12-16
2024-12-06
2024-12-05
2024-12-04
2024-11-15
2024-11-14
2024-11-14
2024-11-13
2024-11-08
2024-11-07
2024-11-06
2024-10-18
2024-10-17
2024-10-16
2024-10-11
2024-10-11
2024-10-10
2024-10-09
2024-09-27
2024-09-26
2024-09-25
2024-09-13
2024-09-12
2024-09-11
2024-09-11
2024-09-10
2024-07-05
2024-07-04
2024-07-03
2024-06-28
2024-06-27
2024-06-26
2024-06-14
2024-06-14

ChunkedEncodingError: ('Connection broken: IncompleteRead(10232 bytes read, 8 more expected)', IncompleteRead(10232 bytes read, 8 more expected))

In [17]:
dates = []
for j in range(len(responses)):
    for i in range(len(responses[j]['documents'])):
        dates.append(responses[j]['documents'][i]['datum'])

In [19]:
f'The dataset ranges from {min(dates)} to {max(dates)}'

'The dataset ranges from 2005-04-20 to 2025-11-25'

In [22]:
n_doc = len(responses[0]['documents'])*len(responses)

In [23]:
f'The dataset contains {n_doc} documents '

'The dataset contains 1380 documents '

In [40]:
type(responses[3]['documents'][0]['text'])

str

Needed to export at least one document to test smth

In [39]:
with open("Output.txt", "w") as text_file:
    text_file.write(responses[3]['documents'][0]['text'])

### Split corpus into the main part (plenary discussion)

In [392]:
#start of discussion
start = responses[0]['documents'][0]['text'].index('Die Sitzung ist eröffnet')
#end of discussion
end = responses[0]['documents'][0]['text'].index('Die Sitzung ist hiermit geschlossen.')

#slice corpus
sitzungs_text = responses[0]['documents'][0]['text'][start:end]

### Cleaning


In [360]:
def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('german')) ## define stopwords

    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]

    #cleaned_sentence = ' '.join(word for word in lemmatized)

    return lemmatized

In [377]:
len(cleaning(responses[0]['documents'][0]['text']))

25608

In [ ]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"],
                                            batch_size=-1, as_supervised=True)

train_sentences, train_labels = tfds.as_numpy(train_data)
test_sentences, test_labels = tfds.as_numpy(test_data)

# Let's check two sentences
train_sentences[0:2]

### Similarity Comparison

In [25]:
!pip install sentence_transformers

  Using cached torch-2.9.1-cp312-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached pillow-12.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 10.0 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 8.3 MB/s  0:00:00
Using cached torch-2.9.1-cp312-none-macosx_11_0_arm64.whl (74.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s  0:00:00m eta 0:00:01
Using cached pillow-12.0.0-cp312-cp312-macosx_11_0_arm64.whl (4.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 8.6 MB/s  0:00:00 eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 10.1 MB/s  0:00:02 eta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: huggingface-hub0m╺━━━━━━━━━━━━━━━  6/10 [scikit-learn]
    Found existing installation: huggingface_hub 1.1.5━━━━━━━━  6/10 [sci

In [26]:
from sentence_transformers import CrossEncoder

# model = CrossEncoder("cross-encoder/nli-deberta-v3-base")
# scores = model.predict([
#     ("A man is eating pizza", "A man eats something"),
#     ("A black race car starts up in front of a crowd of people.", "A man is driving down a lonely road."),
# ])

# # Convert scores to labels
# label_mapping = ["contradiction", "entailment", "neutral"]
# labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]
# # => ['entailment', 'contradiction']

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 